# 시각장애인을 위한 교통 정보 검색 시스템

## 라이브러리 불러오기

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import warnings
import requests
from bs4 import BeautifulSoup
import speech_recognition as sr
from gtts import gTTS
import sounddevice
import os
import time
from scipy.io.wavfile import write
from google.cloud import texttospeech
from google.cloud import speech
import re
import subprocess
from tkinter import *
import tkinter as tk

## 크롬 옵션 및 구동 위한 기타 함수 설정

In [2]:
# 크롬 옵션 객체 생성
options = Options()

# 지정한 user-agent로 설정합니다.
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36")
options.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')
options.add_argument('--start-maximized')

# pref로 옵션 조절(headless 적용 시 사용 불가)
prefs = {'profile.default_content_setting_values': {'cookies'                   : 2, 'images': 1,
                                                    'plugins'                   : 2, 'popups': 2, 'geolocation': 2,
                                                    'notifications'             : 2, 'auto_select_certificate': 2,
                                                    'fullscreen'                : 2,
                                                    'mouselock'                 : 2, 'mixed_script': 2,
                                                    'media_stream'              : 2,
                                                    'media_stream_mic'          : 2, 'media_stream_camera': 2,
                                                    'protocol_handlers'         : 2,
                                                    'ppapi_broker'              : 2, 'automatic_downloads': 2,
                                                    'midi_sysex'                : 2,
                                                    'push_messaging'            : 2, 'ssl_cert_decisions': 2,
                                                    'metro_switch_to_desktop'   : 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2,
                                                    'site_engagement'           : 2,
                                                    'durable_storage'           : 2}}
options.add_experimental_option('prefs', prefs)

# 경고 무시
warnings.filterwarnings(action='ignore')

# 크롬 옵션 객체 생성(BIS 전용)
options_2 = Options()

# 지정한 user-agent로 설정합니다.(BIS 전용)
options_2 = webdriver.ChromeOptions()
options_2.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36")

options_2.add_argument('headless')
options_2.add_argument('disable-gpu')
options_2.add_argument('disable-infobars')
options_2.add_argument('--disable-extensions')
options_2.add_argument('--start-maximized')

# 음성 녹음 함수
def voiceRecog(num):
    r = sr.Recognizer()
    
    while(True) : 
        try : 
        
            with sr.Microphone() as source:
                print('음성을 인식 중입니다.')
                os.system("afplay " + "음성인식_1.mp3")
                audio = r.record(source, duration = num)
                os.system("afplay " + "음성인식_2.mp3")
                print('응답 : ' + r.recognize_google(audio, language='ko-KR'))
                return r.recognize_google(audio, language='ko-KR')
        except sr.UnknownValueError :
                print("죄송합니다. 이후 안내에 따라 다시 한번 말씀해주십시오.")
                os.system("afplay " + "음성인식_3.mp3")
                continue

# 지하철 몇 다시 몇 처리 함수
def convert_format(data):
    converted_data = data.replace("-", "다시")
    return converted_data

## 교통 안내문 제작

In [3]:
# 안내문 입력 함수화
def speak() :
    num = int(input("몇 개의 안내문을 입력하시겠습니까? : "))
    for i in range(num):
        text = input(f"안내문 {i+1}을 입력하세요. : ")
        tts = gTTS(text=text, lang='ko')
        filename = f'안내문_{i+1}.mp3'
        tts.save(filename)
        print(f"안내문 {i+1}이 파일로 저장되었습니다.")

## 교통 정보 검색 시스템 실행 함수

In [4]:
def traffic_service() : 
    print("시각장애인을 위한 교통 정보 검색 시스템을 시작합니다.")
    os.system("afplay " + "안내문_1.mp3")

    # 웹페이지가 로딩될 때까지 5초는 기다림
    driver = webdriver.Chrome('/Users/kanovatio/chromedriver', options=options)

    # 초기화면(네이버 길찾기)
    naver_map = "https://map.naver.com/v5/directions/-/-/-/transit?c=15,0,0,0,dh"

    # 로딩 안내
    print("시스템에 접속 중입니다. 다음 음성 안내를 잠시만 기다려주십시오.")
    os.system("afplay " + "안내문_2.mp3")

    # 웹페이지 해당 주소 이동
    driver.get(naver_map)
    element = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#directionStart0"))
    )

    # 페이지 리부팅 대비
    time.sleep(3)

    # 출발지 입력
    os.system("afplay " + "안내문_3.mp3")
    # starting_point = input("출발지를 말씀하십시오 : ") # 테스트용
    # starting_point = "어린이대공원역" # 테스트용
    starting_point = voiceRecog(5) # 음성인식
    start_place= driver.find_element(By.CSS_SELECTOR, '#directionStart0')
    start_place.send_keys(starting_point)
    driver.implicitly_wait(5)
    WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CLASS_NAME, 'place_text_box.ng-star-inserted'))
    )
    
    # 마우스 클릭이 확실히 되도록 설정
    actions = ActionChains(driver)
    text_boxes = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[1]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container/div/nm-list/ul/li[1]/a/div/div[1]')
    place_elements = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[1]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container/div/nm-list/ul/li[1]/a/span/span')

    # XPATH의 경로에 있어 nm-list-container와 nm-list-container[2]가 존재하기 때문에 예외처리를 통해 검색을 도와줌
    if len(text_boxes) > 0 : 
            if place_elements[0].text == "장소" : 
                actions.move_to_element(text_boxes[0]).click().perform()
            else : 
                text_boxes = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[1]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container[2]/div/nm-list/ul/li[1]/a/div/div[1]')
                place_elements = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[1]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container[2]/div/nm-list/ul/li[1]/a/span/span')
                if len(text_boxes) > 0 : 
                    if place_elements[0].text == "장소" : 
                        actions.move_to_element(text_boxes[0]).click().perform()
                    else : 
                        print("정확한 지명을 확인해주십시오.")
                        
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#container > shrinkable-layout > div > directions-layout > directions-result > div.main > div > directions-search > div.search_box > directions-search-box:nth-child(1) > div > div > span')))
    time.sleep(2)
    
    # 도착지 입력
    os.system("afplay " + "안내문_4.mp3")
    # ending_point = input("도착지를 말씀하십시오 : ") # 테스트용
    # ending_point = "종로3가역" # 테스트용
    ending_point = voiceRecog(5) # 음성인식
    end_place = driver.find_element(By.CSS_SELECTOR, '#directionGoal1')
    end_place.send_keys(ending_point)
    driver.implicitly_wait(5)
    WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CLASS_NAME, 'place_text_box.ng-star-inserted'))
    )
    
    # 마우스 클릭이 확실히 되도록 설정
    text_boxes_2 = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[2]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container/div/nm-list/ul/li[1]/a/div/div[1]')
    place_elements_2 = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[2]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container/div/nm-list/ul/li[1]/a/span/span')

    # XPATH의 경로에 있어 nm-list-container와 nm-list-container[2]가 존재하기 때문에 예외처리를 통해 검색을 도와줌
    if len(text_boxes_2) > 0 : 
            if place_elements_2[0].text == "장소" : 
                actions.move_to_element(text_boxes_2[0]).click().perform()
            else : 
                text_boxes_2 = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[2]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container[2]/div/nm-list/ul/li[1]/a/div/div[1]')
                place_elements_2 = driver.find_elements(By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[2]/div/directions-search-box-instant-search/div/instant-search-list/div/div/nm-list-container[2]/div/nm-list/ul/li[1]/a/span/span')
                if len(text_boxes_2) > 0 : 
                    if place_elements_2[0].text == "장소" : 
                        actions.move_to_element(text_boxes_2[0]).click().perform()
                    else : 
                        print("정확한 지명을 확인해주십시오.")
    
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[1]/directions-search-box[1]/div/div/span')))
    time.sleep(2)
    
    # 경로 탐색(길찾기 버튼 클릭 + 상세정보 클릭)
    print("최적의 추천 경로를 탐색 중입니다.")
    os.system("afplay " + "안내문_5.mp3")

    element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CLASS_NAME, 'btn_detail')))

    # 상세 보기 클릭
    specific_click = driver.find_element(By.CLASS_NAME, 'btn_detail')
    specific_click.click()
    driver.implicitly_wait(5)

    # 교통 정보 스크래이핑
    traffic_info = driver.find_element(By.CLASS_NAME, 'list_path')
    print("-----"*13)
    soup = BeautifulSoup(traffic_info.text, 'html.parser')
    
    # 제거 단어 설정
    words_to_remove = ["파노라마 보기", "거리뷰", "시간표", "상세정보", "이동수단", "대체버스"]

    # 텍스트 추출 후 정보 필터링
    info_text = soup.get_text()
    for word in words_to_remove :
        info_text = info_text.replace(word, "")

    # 음성 정보 필터링
    tts_text = info_text
    processed_text = re.sub(r'\b\d{5}\b', '', tts_text)
    tts_text_p = convert_format(processed_text)
    
    # 처음으로 언급되는 버스 번호를 할당해주기(이후 버스 특정에 사용됨)
    if "정류장" in info_text : 
        bus_name_box = driver.find_elements(By.CLASS_NAME, 'bus_name_box')
        bus_first = bus_name_box[0].text

    # 음성 정보 저장
    traffic_tts = gTTS(text = tts_text_p, lang='ko')
    filename = f'경로 음성 안내_{starting_point}_{ending_point}.wav'
    traffic_tts.save(filename)
    file_path = f"경로 음성 안내_{starting_point}_{ending_point}.wav"
    print("\n")
    print("검색하신 교통 정보에 대해 말씀드리겠습니다.")
    os.system("afplay " + "안내문_6.mp3")
    print(info_text)
    # 교통 정보 재생
    subprocess.call(["afplay", file_path])

    # 정류장 Keyword 존재 시 BIS(Bus Information System) 접속
    if "정류장" in info_text : 
        driver.quit()
        driver = webdriver.Chrome('/Users/kanovatio/chromedriver', options=options_2)
        bis_url = 'https://bus.go.kr/app/#viewpage/1000001/main.nearbusinfo/1/title=Home%20%3E%20%EB%82%B4%EC%A3%BC%EB%B3%80%EC%A0%95%EB%A5%98%EC%86%8C'
        driver.get(bis_url)
        print("-----"*13)
        print("경로상 존재하는 버스 정보를 안내해드리기 위해 버스 정보 시스템에 접속합니다.")
        os.system("afplay " + "안내문_7.mp3")
        search_box = driver.find_element(By.CLASS_NAME, 'x-input-el')
        search_box.click()
        serial_number = re.findall(r'\b\d{5}\b', info_text)
        station_serial_number = serial_number[0]
        search_box.send_keys(station_serial_number)
        search_box.send_keys(Keys.RETURN)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ext-list-2')))

        # 버스 교통 정보 필터링 및 음성 출력
        bus_list = driver.find_element(By.CSS_SELECTOR, '#ext-list-2')
        print("검색하신 버스에 대한 자세한 정보를 말씀드리겠습니다.")
        os.system("afplay " + "안내문_bus_1.mp3")
        time.sleep(0.5)
        end_pattern = r"막차시간:\s*(\d{2}:\d{2})"
        # 정규 표현식 패턴을 사용하여 해당 버스의 번호를 이용한 정보 추출
        pattern = re.escape(bus_first) + r"\b.*?" + end_pattern
        match = re.search(pattern, bus_list.text, re.DOTALL)

        if match :
            extracted_info = match.group(0)
            extracted_info = extracted_info.replace("~", "부터 ")
            bus_tts = gTTS(text = extracted_info, lang='ko')
            filename = f'버스상세정보_{bus_first}번.wav'
            bus_tts.save(filename)
            file_path_bus = f'버스상세정보_{bus_first}번.wav'
            print("버스 정보 : ", extracted_info)
            subprocess.call(["afplay", file_path_bus])
            
        else :
            print("해당 버스에 대한 정보를 찾을 수 없습니다. 다음 안내로 이동합니다.")
            os.system("afplay " + "안내문_bus_2.mp3")
            pass
        
    else : 
        pass

    # 다시 듣기 서비스    
    print("-----"*13)
    print('다시 한번 듣고 싶으시다면 "예", 이대로 종료를 원하신다면 "아니오"를 말씀해주십시오.')
    os.system("afplay " + "안내문_8.mp3")
    answer = voiceRecog(5)
    # answer = "아니오" # 테스트용
    if answer == "예" or answer == "네" : 
        print("검색하신 교통 정보를 다시 한번 말씀드리겠습니다.")
        os.system("afplay " + "안내문_9.mp3")
        time.sleep(1)
        # 교통 정보 다시 재생
        subprocess.call(["afplay", file_path])
        time.sleep(1)
        print("서비스를 종료합니다. 이용해주셔서 감사합니다.")
        os.system("afplay " + "안내문_10.mp3")
        # 웹드라이버 종료
        driver.quit()

    else : 
        print('서비스를 종료합니다. 이용해주셔서 감사합니다.')
        os.system("afplay " + "안내문_10.mp3")
        # 웹드라이버 종료
        driver.quit()

In [5]:
# 함수 구동
# traffic_service()

# 시각장애인을 위한 경제 뉴스 요약 및 리딩 서비스

## 라이브러리 불러오기

In [6]:
from bs4 import BeautifulSoup     
import requests
import re
from konlpy.tag import Kkma
from konlpy.tag import Twitter   # Okt로 하면 오류남. 그래서 Twitter로 대체함
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
from datetime import datetime

## 뉴스 안내문 제작

In [7]:
# 뉴스 안내문 제작
def news_speak() :
    num = int(input("몇 개의 안내문을 입력하시겠습니까? : "))
    for i in range(num):
        text = input(f"뉴스안내문 {i}을 입력하세요. : ")
        tts = gTTS(text=text, lang='ko')
        filename = f'뉴스안내문_{i}.mp3'
        tts.save(filename)
        print(f"뉴스안내문_{i}이 파일로 저장되었습니다.")

In [8]:
def eco_news() : 
    print("시각장애인을 위한 경제 뉴스 요약 및 리딩 서비스를 시작합니다.")
    os.system("afplay " + "뉴스안내문_0.mp3")
    print("간결한 뉴스 청취를 희망하신다면 '간결', 자세한 뉴스 청취를 희망하신다면 '자세'라고 응답해주십시오.")
    os.system("afplay " + "뉴스안내문_1.mp3")
    # 테스트용
    # record = "간결"

    # 문장 요약 수
    while(True) : 
        record = voiceRecog(5)
        if record == "간결" :
            number = 3
            break
        elif record == "상세" : 
            number = 5
            break
        else : 
            print("죄송합니다. 다시 한번 말씀해주십시오.")
            os.system("afplay " + "뉴스안내문_2.mp3")

    # 실시간 기사 개수 선택
    print("25개의 최신 기사를 불러왔습니다. 몇 번째 기사를 청취하시겠습니까? 서수로 말씀해주십시오.")
    os.system("afplay " + "뉴스안내문_3.mp3")
    # record = "셋" # 테스트용
    # record = voiceRecog(5)
    # ranking이라고 표현했으나 위에서 몇 번째인지를 의미하는 객체입니다.
    while(True) : 
        record = voiceRecog(5)
        if record == "하나" or record == '1' : 
            ranking = 1
            break
        elif record == "둘" or record == '2' : 
            ranking = 2
            break
        elif record == "셋" or record == '3' : 
            ranking = 3
            break
        elif record == "넷" or record == '4' : 
            ranking = 4
            break
        elif record == "다섯" or record == '5' : 
            ranking = 5
            break
        elif record == "여섯" or record == '6' : 
            ranking = 6
            break
        elif record == "일곱" or record == '7' : 
            ranking = 7
            break
        elif record == "여덟" or record == '8' : 
            ranking = 8
            break
        elif record == "아홉" or record == '9' : 
            ranking = 9
            break
        elif record == "열" or record == '10' : 
            ranking = 10
            break
        elif record == "열하나" or record == '11' : 
            ranking = 11
            break
        elif record == "열둘" or record == '12' : 
            ranking = 12
            break
        elif record == "열셋" or record == '13' : 
            ranking = 13
            break
        elif record == "열넷" or record == '14' : 
            ranking = 14
            break
        elif record == "열다섯" or record == '15' : 
            ranking = 15
            break
        elif record == "열여섯" or record == '16' : 
            ranking = 16
            break
        elif record == "열일곱" or record == '17' : 
            ranking = 17
            break
        elif record == "열여덟" or record == '18' : 
            ranking = 18
            break
        elif record == "열아홉" or record == '19' : 
            ranking = 19
            break
        elif record == "스물" or record == '20' : 
            ranking = 20
            break
        elif record == "스물하나" or record == '21' : 
            ranking = 21
            break
        elif record == "스물둘" or record == '22' : 
            ranking = 22
            break
        elif record == "스물셋" or record == '23' : 
            ranking = 23
            break
        elif record == "스물넷" or record == '24' : 
            ranking = 24
            break
        elif record == "스물다섯" or record == '25' : 
            ranking = 25
            break
        else : 
            print("죄송합니다. 다시 한번 말씀해주십시오. 참고로 서수는 숫자를 하나, 둘, 셋으로 읽는 방법을 말합니다.")
            os.system("afplay " + "뉴스안내문_2_2.mp3")

    # 뉴스 크롤링 시작
    print("작업을 시작합니다. 잠시만 기다려주십시오.")
    os.system("afplay " + "뉴스안내문_3_1.mp3")        
    print("---------"*13)

    #웹페이지 스크립트 가져오기
    url = "https://finance.naver.com/news/news_list.nhn?"  
    params = {'mode' : 'RANK'}


    resp = requests.get(url, params) #합쳐서 날짜, 순위에 맞는 링크 추출됨.
    # print(resp)

    soup = BeautifulSoup(resp.content, 'lxml')


    #각 뉴스기사의 링크 크롤링하기
    url_list = soup.find('div','hotNewsList').find_all('a')
    news_url_list = []

    for url in url_list :
        news_url_list.append('https://finance.naver.com' + url['href'])

    # print(len(news_url_list)) #25출력 => 한 페이지당 25개의 뉴스기사 존재
    news_url_list[0]

    # 링크가 주어지면, 기사 제목과 본문을 크롤링하는 코드
    def get_news_by_url(url):
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'} # 각자 본인의 User-Agent 정보 입력 
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.content, 'html.parser')

        #제목
        title = bs.find('div','article_info').find('h3').string
        title = re.sub('\t|\r|\n| ', '', title)
        #본문
        content = bs.find('div','articleCont').text
        content = re.sub('\xa0|\t|\r|\n|', '', content)
        return  title + content

    #25개의 url을 통해 각 뉴스기사 페이지의 본문을 가져온다!
    news_content = [0]
    for url in news_url_list :
        news_content.append(get_news_by_url(url))

    #한글분석기
    kkma = Kkma() 

    #text를 입력받아 Kkma.sentences()를 이용해 문장단위로 나눈 뒤 sentences로 리턴
    def text2sentences(text):  
        sentences = kkma.sentences(text)  #text일 때 문장별로 리스트 만듦
        for idx in range(0, len(sentences)):  #길이에 따라 문장 합침(위와 동일)
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences


    #기사본문 가져오기(리스트형태로)
    sentences = text2sentences(news_content[ranking])

    twitter = Twitter()

    #불용어제거
    stopwords = ['머니투데이' , "연합뉴스", "데일리", "동아일보", "중앙일보", 
                 "조선일보", "기자","아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", 
                 "저희", "따라", "의해", "을", "를", "에", "의", "가"]

    def get_nouns(sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                      if noun not in stopwords and len(noun) > 1]))
        return nouns


    nouns = get_nouns(sentences)

    tfidf = TfidfVectorizer()
    cnt_vec = CountVectorizer()
    graph_sentence = []

    def build_sent_graph(sentence):
        tfidf_mat = tfidf.fit_transform(sentence).toarray()
        graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return graph_sentence

    sent_graph = build_sent_graph(nouns)

    def build_words_graph(sentence):
        cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

    words_graph, idx2word = build_words_graph(nouns)

    def get_ranks(graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

    sent_rank_idx = get_ranks(sent_graph)  #sent_graph : sentence 가중치
    # sent_rank_idx


    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)

    word_rank_idx = get_ranks(words_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

    # 문장 수에 맞는 기사 요약 추출
    def summarize(sent_num=5):
        summary = []
        index=[]
        for idx in sorted_sent_rank_idx[:sent_num]:
            index.append(idx)

        index.sort()

        for idx in index:
            summary.append(sentences[idx])
        for text in summary :
            print(text)
            print("\n")    
        result = "\n".join(summary)
        return result

        for text in summary :
            print(text)
            print("\n")

    # summarize(number)

    def keywords(word_num=10):

        keywords = []
        index=[]
        for idx in sorted_word_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(idx2word[idx])

        print(keywords)

    # keywords()

    print('\n', '< 기사 본문 링크 >')
    print(news_url_list[ranking - 1])

    print('\n', '< 핵심어 >')
    keywords()

    print('\n', '< 요약 >')
    text_summary = summarize(number)
    text_summary = gTTS(text = text_summary, lang='ko')
    filename = '뉴스요약.wav'
    text_summary.save(filename)
    file_path = "뉴스요약.wav"
    subprocess.call(["afplay", file_path])

    print("---------"*13)
    print('다시 한번 듣고 싶으시다면 "예", 이대로 종료를 원하신다면 "아니오"를 말씀해주십시오.')
    os.system("afplay " + "뉴스안내문_4.mp3")
    YorN = voiceRecog(5)
    # YorN = "아니오" # 테스트용
    if YorN == "예" or YorN == "네" : 
        print("검색하신 경제 뉴스 정보를 다시 한번 말씀드리겠습니다.")
        os.system("afplay " + "뉴스안내문_5.mp3")
        time.sleep(1)
        # 교통 정보 다시 재생
        subprocess.call(["afplay", file_path])
        time.sleep(1)
        print("서비스를 종료합니다. 이용해주셔서 감사합니다.")
        os.system("afplay " + "뉴스안내문_6.mp3")
        time.sleep(0.5)

    else : 
        print('서비스를 종료합니다. 이용해주셔서 감사합니다.')
        os.system("afplay " + "뉴스안내문_6.mp3")
        time.sleep(0.5)

In [9]:
# 뉴스 크롤링 및 요약 함수 실행
# eco_news()

# GUI 구현

In [11]:
def start_application():
    print("시각장애인을 위한 프로그램을 실행합니다. '교통' 혹은 '경제' 중 원하시는 서비스를 말씀해주십시오.")
    os.system("afplay " + "안내문_0_1_1.mp3")
    voice_result = voiceRecog(5)
    if voice_result == "교통":
        time.sleep(1)
        traffic_service()
    elif voice_result == "경제":
        time.sleep(1)
        eco_news()
    else:
        print("이용해주셔서 감사합니다.")
        os.system("afplay " + "안내문_0_2.mp3")
        root.quit()

# 종료 기능
def quit_application():
    root.destroy()

root = Tk()
root.title("시각장애인을 위한 음성 안내 시스템")

# 시작 버튼
start_button = tk.Button(root, text="시스템 구동", command=start_application, width=8, height=1)
start_button.pack()

# 종료 버튼
quit_button = tk.Button(root, text="종료", command=quit_application, width=8, height=1)
quit_button.pack()

# 윈도우 크기 설정
window_width = 320
window_height = 80

# 윈도우 위치 계산
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2

# 윈도우 위치 및 크기 설정
root.geometry(f"{window_width}x{window_height}+{x}+{y}")

# 버튼을 인터페이스의 중앙에 배치
start_button.place(relx=0.5, rely=0.3, anchor=tk.CENTER)
quit_button.place(relx=0.5, rely=0.7, anchor=tk.CENTER)

# 프로그램 실행
root.mainloop()

시각장애인을 위한 프로그램을 실행합니다. '교통' 혹은 '경제' 중 원하시는 서비스를 말씀해주십시오.
음성을 인식 중입니다.
응답 : 교통
시각장애인을 위한 교통 정보 검색 시스템을 시작합니다.
시스템에 접속 중입니다. 다음 음성 안내를 잠시만 기다려주십시오.
음성을 인식 중입니다.
응답 : 건대입구역
음성을 인식 중입니다.
응답 : 뚝섬 유원지역
최적의 추천 경로를 탐색 중입니다.
-----------------------------------------------------------------


검색하신 교통 정보에 대해 말씀드리겠습니다.
12:08
출발
건대입구역 2호선

4분
도보 243m
지하철
2분
7호선 건대입구역 승차
뚝섬유원지역 방면
빠른하차 2-3, 6-2
12:11 석남행
1개 역 이동

뚝섬유원지역 7호선 하차
내리는 문: 오른쪽
12:13
도착
뚝섬유원지역 7호선

-----------------------------------------------------------------
다시 한번 듣고 싶으시다면 "예", 이대로 종료를 원하신다면 "아니오"를 말씀해주십시오.
음성을 인식 중입니다.
응답 : 예
검색하신 교통 정보를 다시 한번 말씀드리겠습니다.
서비스를 종료합니다. 이용해주셔서 감사합니다.
